## Busca de Documentos

In [32]:
import spacy
import json

TEXTS_TO_CATALOG = [
    'O Nubank anunciou um lucro recorde no último trimestre, superando as expectativas de Wall Street. O CEO David Vélez comemorou os resultados.',
    'A Magazine Luiza informou que vai fechar parcerias logísticas no Nordeste. A empresa busca competir com a Amazon na região.',
    'A B3, bolsa de valores brasileira, fechou em queda de 1,5% nesta terça-feira. O Ibovespa foi puxado pelas ações da Vale.',
    'A fábrica da BYD em Camaçari deve começar a operar em breve. O governo da Bahia prometeu incentivos fiscais para a montadora chinesa.',
    'O iFood enfrenta novos processos trabalhistas em São Paulo. Entregadores pedem vínculo empregatício com a plataforma.',
    'O Flamengo venceu o Fluminense no Maracanã por 2 a 0. O atacante Gabigol marcou os dois gols da partida.',
    'Neymar Jr. deve voltar aos campos em breve pelo Al-Hilal. O jogador fez exames em uma clínica de Belo Horizonte.',
    'O Palmeiras anunciou a contratação de um novo zagueiro argentino. A presidente Leila Pereira confirmou a negociação.',
    'O ministro Alexandre de Moraes, do STF, determinou novas diligências. A Polícia Federal cumpriu mandados em Brasília e Curitiba.',
    'O Senado Federal aprovou a nova lei de diretrizes orçamentárias. O presidente do Senado, Rodrigo Pacheco, elogiou a votação.',
    'O governador Tarcísio de Freitas inaugurou uma nova estação de trem. O evento ocorreu na zona leste de São Paulo.',
    'A colheita de soja no Mato Grosso atingiu 80% da área plantada. A Embrapa alertou para os riscos climáticos na região.',
    'O Ibama multou madeireiras ilegais no sul do Pará. A ministra Marina Silva reforçou a importância da fiscalização na Amazônia.',
    'A JBS anunciou a compra de uma nova planta de processamento nos Estados Unidos. A empresa brasileira expande sua atuação global.',
    'A cantora Anitta confirmou um show gratuito na praia de Copacabana. A prefeitura do Rio de Janeiro espera 1 milhão de pessoas.',
    'O Rock in Rio anunciou as atrações principais do palco mundo. A venda de ingressos começa na próxima semana pela Ticketmaster.',
    'A Fiocruz entregou um novo lote de vacinas contra a dengue ao Ministério da Saúde. A distribuição começa pelo Distrito Federal.',
    'Pesquisadores da USP descobriram uma nova molécula promissora. O estudo foi publicado em parceria com a Fapesp.',
    'Um forte temporal atingiu Porto Alegre ontem à noite. A Defesa Civil emitiu alertas para o interior do Rio Grande do Sul.',
    'A Embraer fechou um contrato bilionário para venda de jatos comerciais. A entrega será feita para uma companhia aérea da Europa.'
]

document_database = []

def catalog_documents(text_list, model_path="model_news2"):
    print(f"Carregando modelo customizado de '{model_path}'...")
    try:
        nlp = spacy.load('model_news2')
    except IOError:
        print(f"--- ERRO ---")
        print(f"Modelo '{model_path}' não encontrado.")
        return None
    
    print("Modelo carregado. Iniciando catalogação...\n")
    
    docs = nlp.pipe(text_list)
    
    db = []
    for i, doc in enumerate(docs):
        entities = []
        for ent in doc.ents:
            entities.append({
                "text": ent.text,
                "label": ent.label_
            })
        
        db_entry = {
            "id": f"doc_{i+1}",
            "text": doc.text,
            "entities": entities
        }
        db.append(db_entry)
        
    print(f"Catalogação concluída! {len(db)} documentos indexados.")
    return db

In [33]:
def search_documents(db, query):
    results = []
    
    print(f"\n---Iniciando Busca ---")
    print(f"Query: {query}")
    
    for entry in db:
        match = True  
        
        for query_label, query_text in query.items():
            
            found_entity_match = False 
            
            for entity in entry['entities']:
                if (entity['label'] == query_label and 
                    entity['text'].lower() == query_text.lower()):
                    
                    found_entity_match = True
                    break 
            
            if not found_entity_match:
                match = False
                break 
        
        if match:
            results.append(entry)
            
    return results

In [34]:
def print_results(results):
    if not results:
        print("Nenhum resultado encontrado.")
        return
        
    total = len(results)
    print(f"\nEncontrados {total} resultados:\n")
    print("=" * 60)

    for i, res in enumerate(results, 1):
        if isinstance(res['entities'], list):
            entidades_str = ", ".join([str(e) for e in res['entities']])
        else:
            entidades_str = str(res['entities'])
        
        print(f"RESULTADO {i}/{total} | ID: {res['id']}")
        print("-" * 60)
        print(f"Texto:\n   {res['text'][:150]}...") 
        print(f"\nEntidades: {entidades_str}")
        print("=" * 60)

In [35]:
def save_to_json(data, filename="relatorio_buscas.json"):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"\nRelatório completo salvo em: {filename}")


document_database = catalog_documents(TEXTS_TO_CATALOG)

if document_database:
    
    relatorio_final = []

    # BUSCA 1: Ibama
    print("\n--- Buscando ORG: Ibama ---")
    query1 = {"ORG": "Ibama", "LOC": "Amazônia"}
    results1 = search_documents(document_database, query1)
    print_results(results1)
    
    relatorio_final.append({
        "id_busca": 1,
        "criterios": query1,
        "total_encontrado": len(results1),
        "resultados": results1
    })

    # BUSCA 2: Pessoa (Tarcísio) e Loc(São Paulo)
    print("\n--- Buscando Per: Tarcísio ---")
    query2 = {"PER": "Tarcísio de Freitas", "LOC": "São Paulo"}
    results2 = search_documents(document_database, query2)
    print_results(results2)

    relatorio_final.append({
        "id_busca": 2,
        "criterios": query2,
        "total_encontrado": len(results2),
        "resultados": results2
    })

    # BUSCA 3: Agro/Governo (Embrapa)
    print("\n--- Buscando ORG: Embrapa ---")
    query3 = {"ORG": "Embrapa"}
    results3 = search_documents(document_database, query3)
    print_results(results3)

    relatorio_final.append({
        "id_busca": 3,
        "criterios": query3,
        "total_encontrado": len(results3),
        "resultados": results3
    })

    # BUSCA 4: Judiciário (Alexandre de Moraes)
    print("\n--- Buscando PER: Alexandre de Moraes ---")
    query4 = {"PER": "Alexandre de Moraes"}
    results4 = search_documents(document_database, query4)
    print_results(results4)

    relatorio_final.append({
        "id_busca": 4,
        "criterios": query4,
        "total_encontrado": len(results4),
        "resultados": results4
    })

    # Salva tudo no arquivo final
    save_to_json(relatorio_final, "resultados_analise_noticias.json")

Carregando modelo customizado de 'model_news2'...
Modelo carregado. Iniciando catalogação...

Catalogação concluída! 20 documentos indexados.

--- Buscando ORG: Ibama ---

---Iniciando Busca ---
Query: {'ORG': 'Ibama', 'LOC': 'Amazônia'}

Encontrados 1 resultados:

RESULTADO 1/1 | ID: doc_13
------------------------------------------------------------
Texto:
   O Ibama multou madeireiras ilegais no sul do Pará. A ministra Marina Silva reforçou a importância da fiscalização na Amazônia....

Entidades: {'text': 'Ibama', 'label': 'ORG'}, {'text': 'Pará', 'label': 'LOC'}, {'text': 'ministra', 'label': 'CARGO'}, {'text': 'Marina Silva', 'label': 'PER'}, {'text': 'fiscalização', 'label': 'TEMA'}, {'text': 'Amazônia', 'label': 'LOC'}

--- Buscando Per: Tarcísio ---

---Iniciando Busca ---
Query: {'PER': 'Tarcísio de Freitas', 'LOC': 'São Paulo'}

Encontrados 1 resultados:

RESULTADO 1/1 | ID: doc_11
------------------------------------------------------------
Texto:
   O governador Tarcísio d